# Thư viện

In [1]:
import polars as pl
import numpy as np
import datetime

# Tải dữ liệu

In [ ]:
user_path = "processed_data/processed_user.parquet"
item_path = "processed_data/processed_item.parquet"
purchase_path = "processed_data/processed_purchase.parquet"

user_df = pl.scan_parquet(user_path)
item_df = pl.scan_parquet(item_path)
purchase_df = pl.scan_parquet(purchase_path)

# Hàm tạo dữ liệu training

In [4]:
def createTrainData(user_df: pl.LazyFrame, item_df: pl.LazyFrame, purchase_df: pl.LazyFrame, begin_hist, end_hist, begin_recent, end_recent, k, seed):
    # Nhãn positive
    positive_df = (
        purchase_df
        .filter(
            pl.col('datetime').is_between(begin_recent, end_recent)
        )
        .unique(
            subset=['customer_id', 'item_id'],
            keep='first'
        )
        .with_columns(
            pl.lit(1).alias('Y')
        )
        .select(
            pl.col(['customer_id', 'item_id', 'Y'])
        )
    )
    # Số mẫu positive
    n_positive = positive_df.select(pl.len()).collect().item()

    # Các ứng cử cho nhãn negative
    negative_candidates_df = (
        purchase_df
        .filter(
            pl.col('datetime').is_between(begin_hist, end_hist)
        )
        .unique(subset=['customer_id', 'item_id'], keep='first')
        .join(
            positive_df.select(['customer_id', 'item_id']),
            on=['customer_id', 'item_id'],
            how='anti'
        )
        .select(
            pl.col(['customer_id', 'item_id'])
        )
        .sort(['customer_id', 'item_id'])
    )
    # Nhãn negative với số mẫu gấp k lần positive
    n_negative_candidates = negative_candidates_df.select(pl.len()).collect().item()
    # Chọn random theo seed
    np.random.seed(seed)
    indices = sorted(np.random.choice(n_negative_candidates, size=k*n_positive, replace=False))
    # Nhãn negative
    negative_df = (
        negative_candidates_df
        .with_columns(
            pl.lit(0).alias('Y')   
        )
        .select(
            pl.all().gather(indices)
        )
    )

#-------------------------------------------------------------------------------------------------------------------------

    # Gom negative và positive
    base_df = pl.concat(items=[positive_df, negative_df])

    # Lọc purchase_df theo cửa sổ thời gian
    hist_filtered_df = purchase_df.filter(
        pl.col("datetime").is_between(begin_hist, end_hist)
    )

    # Join với item_df để lấy thuộc tính của các item đã mua
    hist_with_attrs_df = hist_filtered_df.join(item_df, on="item_id")

    # Đếm số lần xuất hiện brand
    brand_counts = hist_with_attrs_df.group_by("customer_id", "brand").len().rename({"len": "brand_count"})

    # Đếm số lần xuất hiện age_group
    age_group_counts = hist_with_attrs_df.group_by("customer_id", "age_group").len().rename({"len": "age_group_count"})

    # Đếm số lần xuất hiện category
    category_counts = hist_with_attrs_df.group_by("customer_id", "category_l1").len().rename({"len": "category_count"})

    # Lấy thuộc tính (brand, age_group, category) cho các item trong base_df
    base_with_attrs = base_df.join(item_df, on="item_id")

    # Join các đặc trưng vào base_df
    train_df = base_with_attrs.join(
        brand_counts, on=["customer_id", "brand"], how="left"
    ).join(
        age_group_counts, on=["customer_id", "age_group"], how="left"
    ).join(
        category_counts, on=["customer_id", "category_l1"], how="left"
    )

    # Điền 0 cho các giá trị null
    # (Trường hợp không có trong lịch sử -> count là 0)
    train_df = train_df.with_columns([
        pl.col("brand_count").fill_null(0),
        pl.col("age_group_count").fill_null(0),
        pl.col("category_count").fill_null(0)
    ])

    train_df.select(
        pl.col(['customer_id', 'item_id', 'brand_count', 'age_group_count', 'category_count', 'Y'])
    ).sink_parquet("train_data/train.parquet")


# Tạo dữ liệu train với k = 3, seed = 42
### Nhãn negative gấp 3 lần positive
### Nhãn positive từ tháng 11-12/2024

In [ ]:
begin_hist = datetime.datetime(
    day=1,
    month=1,
    year=2024
)
end_hist = datetime.datetime(
    day=31,
    month=10,
    year=2024
)
begin_recent = datetime.datetime(
    day=1,
    month=11,
    year=2024
)
end_recent = datetime.datetime(
    day=31,
    month=12,
    year=2024
)

createTrainData(user_df, item_df, purchase_df, begin_hist, end_hist, begin_recent, end_recent, k=3, seed=42)

# Xem thử dữ liệu

In [10]:
train_df = pl.scan_parquet("train_data/train.parquet")

In [11]:
train_df.filter(
    pl.col('Y') == 1
).limit(5).collect()

customer_id,item_id,brand_count,age_group_count,category_count,Y
i32,str,u32,u32,u32,i32
6592738,"""0029040000019""",2,3,0,1
384358,"""3856000000001""",58,62,20,1
1079321,"""0009010040178""",0,0,0,1
8119082,"""7065000000003""",0,0,0,1
1632539,"""7219000000002""",176,118,12,1


In [12]:
train_df.filter(
    pl.col('Y') == 0
).limit(5).collect()

customer_id,item_id,brand_count,age_group_count,category_count,Y
i32,str,u32,u32,u32,i32
437003,"""2041000000005""",6,1,11,0
465735,"""7337000000003""",10,1,2,0
471593,"""0952000000019""",2,7,2,0
473526,"""2609000000001""",15,23,23,0
432661,"""6996000000006""",8,1,4,0
